## 구글 10년 주가 데이터 세트
구글 주식 내역 [2013-01-01]부터 [2024-01-10]까지

LSTM DL을 사용한 Google 주가 예측

In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

In [4]:
df = pd.read_csv('/home/leejieun/workspace/kaggle-notebooks/datasets/jillanisofttech/google-10-years-stockprice-dataset/GoogleStock_Dataset_V2.csv')

In [5]:
df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume
0,0,2013-01-02,17.918339,18.107130,17.846855,18.013729,102033017
1,1,2013-01-03,18.055573,18.229919,17.950716,18.024191,93075567
2,2,2013-01-04,18.165413,18.467529,18.124067,18.380356,110954331
3,3,2013-01-07,18.317591,18.415474,18.196297,18.300158,66476239
4,4,2013-01-08,18.319834,18.338762,18.043119,18.264042,67295297


In [6]:
df.shape

(2775, 7)

In [8]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2770    False
2771    False
2772    False
2773    False
2774    False
Length: 2775, dtype: bool

In [9]:
df = df.loc[~df.index.duplicated(keep='first')]

In [10]:
print('Total number of days present in the dataset: ', df.shape[0])
print('Total number of fields present in the dataset:', df.shape[1])

Total number of days present in the dataset:  2775
Total number of fields present in the dataset: 7


In [11]:
df.shape

(2775, 7)

In [12]:
df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume
0,0,2013-01-02,17.918339,18.107130,17.846855,18.013729,102033017
1,1,2013-01-03,18.055573,18.229919,17.950716,18.024191,93075567
2,2,2013-01-04,18.165413,18.467529,18.124067,18.380356,110954331
3,3,2013-01-07,18.317591,18.415474,18.196297,18.300158,66476239
4,4,2013-01-08,18.319834,18.338762,18.043119,18.264042,67295297


In [13]:
df

,Unnamed: 0,Date,Open,High,Low,Close,Volume
0,0,2013-01-02,17.918339,18.107130,17.846855,18.013729,102033017
1,1,2013-01-03,18.055573,18.229919,17.950716,18.024191,93075567
2,2,2013-01-04,18.165413,18.467529,18.124067,18.380356,110954331
3,3,2013-01-07,18.317591,18.415474,18.196297,18.300158,66476239
4,4,2013-01-08,18.319834,18.338762,18.043119,18.264042,67295297
...,...,...,...,...,...,...,...
2770,2770,2024-01-04,139.850006,140.634995,138.009995,138.039993,18253300
2771,2771,2024-01-05,138.352005,138.809998,136.850006,137.389999,15433200
2772,2772,2024-01-08,138.000000,140.639999,137.880005,140.529999,17645300
2773,2773,2024-01-09,140.059998,142.800003,139.789993,142.559998,19579700


In [14]:
df.describe()

,Unnamed: 0,Open,High,Low,Close,Volume
count,2775.000000,2775.000000,2775.000000,2775.000000,2775.000000,2.775000e+03
mean,1387.000000,64.913671,65.600115,64.273645,64.952090,3.828731e+07
std,801.217823,38.164394,38.630869,37.753744,38.194190,2.612174e+07
min,0.000000,17.550716,17.567652,17.323069,17.506132,1.584340e+05
25%,693.500000,32.755501,33.153002,32.346251,32.710751,2.356953e+07
50%,1387.000000,54.200001,54.778500,53.616100,54.249500,3.023200e+07
75%,2080.500000,94.466000,96.004997,93.208252,94.619999,4.305293e+07
max,2774.000000,151.863495,152.100006,149.887497,150.709000,4.643901e+08


In [15]:
print('Null Values:', df.isnull().values.sum())

Null Values: 0


In [16]:
print('NA values:', df.isnull().values.any())

NA values: False


In [17]:
df = df.dropna()
print('Null Values:', df.isnull().values.sum())
print('NA values:', df.isnull().values.any())

Null Values: 0
NA values: False


In [18]:
sd = df.iloc[0][0]
ed = df.iloc[-1][0]
print('Starting Date', sd)
print('Ending Date', ed)

Starting Date 0
Ending Date 2774


/tmp/ipykernel_1051674/476303387.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_1051674/476303387.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [29]:
df.loc[0,'Open'] # location

17.918338775634766

In [27]:
df.iloc[0,3] # 두번째 속성을 정수를 통해 가져온다

18.10713005065918

In [30]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

y_overall= df.loc[(df['Date'] >= '2013-01-01') &
                  (df['Date'] <= '2023-12-29')]

y_overall.drop(y_overall[['Close', 'Volume']], axis=1)

,Unnamed: 0,Date,Open,High,Low
0,0,2013-01-02,17.918339,18.107130,17.846855
1,1,2013-01-03,18.055573,18.229919,17.950716
2,2,2013-01-04,18.165413,18.467529,18.124067
3,3,2013-01-07,18.317591,18.415474,18.196297
4,4,2013-01-08,18.319834,18.338762,18.043119
...,...,...,...,...,...
2763,2763,2023-12-22,142.130005,143.250000,142.054993
2764,2764,2023-12-26,142.979996,143.945007,142.500000
2765,2765,2023-12-27,142.830002,143.320007,141.050995
2766,2766,2023-12-28,141.850006,142.270004,140.828003


In [33]:
monthvise = y_overall.groupby(y_overall['Date'].dt.strftime('%B'))[['Open', 'Close']].mean() #strftime('%B') : 영어로된 월 문자열
new_order = ['January', 'February', 'March', 'April','May', 'June', 'July', 'August', 'September', 'October', 'Novenber','December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

,Open,Close
Date,,
January,58.753392,58.837130
February,60.982993,60.981546
March,61.519489,61.574183
April,60.900336,60.844471
May,62.008472,62.163371
June,64.051431,64.071185
July,65.149167,65.293906
August,68.825057,68.884898
September,67.848063,67.754696


In [38]:
names = cycle(['Stock Open Price', 'Stock Close Price', 'Stock High Price', 'Stock Low Price'])
fig = px.line(y_overall, x=y_overall.Date, y=[y_overall['Open'], y_overall['Close'],
                                              y_overall['High'], y_overall['Low']],
              labels={'Date': 'Date','value': 'Stock value'})
fig.update_layout(title_text= 'Google Stock analysis chart',font_size= 15, font_color ='black', legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t: t.update(name = next(names)))
fig.update_xaxes(showgrid= False)
fig.update_yaxes(showgrid=False)

fig.show()